In [1]:
from ortools.sat.python import cp_model
from datetime import datetime
import pandas as pd
from openpyxl import load_workbook
import re
import shutil


def extrair_numero(s):
    return int(re.search(r'\d+', s).group())


def carregar_dados(chamados_path, liberacao_path):
    chamados_df = pd.read_excel(chamados_path)
    lib_df = pd.read_excel(liberacao_path)
    return chamados_df, lib_df


def obter_clientes_sem_tecnico(lib_df, clientes):
    # Filtra clientes que não têm técnicos liberados
    clientes_sem_tecnico = lib_df[
        lib_df['cliente'].isin(clientes)
    ].set_index('cliente').apply(
        lambda row: all(str(val).strip().lower() != 'liberado' for val in row),
        axis=1
    )
    return clientes_sem_tecnico[clientes_sem_tecnico].index.tolist()


def separar_chamados_por_tecnico(chamados_df, clientes_sem_tecnico):
    chamados_sem_tecnico = chamados_df[chamados_df['cliente'].isin(clientes_sem_tecnico)].copy()
    chamados_com_tecnico = chamados_df[~chamados_df['cliente'].isin(clientes_sem_tecnico)].copy()
    return chamados_com_tecnico, chamados_sem_tecnico


def atualizar_chamados_excel(chamados_com_tecnico, path='chamados.xlsx'):
    chamados_com_tecnico.reset_index(drop=True, inplace=True)
    chamados_com_tecnico.to_excel(path, index=False)


def atualizar_chamados_nao_agendados_excel(chamados_sem_tecnico, path='chamados_nao_agendados.xlsx'):
    wb = load_workbook(path)
    ws = wb.active

    row_count = 1
    for row in ws.iter_rows(min_row=2, values_only=True):
        if any(cell is not None for cell in row):
            row_count += 1
        else:
            break

    start_row = row_count

    with pd.ExcelWriter(path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        chamados_sem_tecnico.to_excel(writer, index=False, header=False, startrow=start_row)


def obter_tecnicos_validos(lib_df, clientes_validos):
    lib_validos = lib_df[lib_df['cliente'].isin(clientes_validos)].copy()
    tecnicos_validos = []
    for tecnico in lib_validos.columns[1:]:  # Ignora coluna 'cliente'
        if lib_validos[tecnico].astype(str).str.strip().str.lower().eq('liberado').any():
            tecnicos_validos.append(tecnico)
    tecnicos_validos = sorted(tecnicos_validos, key=extrair_numero)
    return tecnicos_validos

def obter_todos_os_tecnicos_possiveis(lib_df):
    # Retorna todas as colunas, exceto a de cliente
    return list(lib_df.columns.drop('cliente'))


def carregar_agendas(tecnicos_validos, arquivo_agenda='agenda.xlsx'):
    xls = pd.ExcelFile(arquivo_agenda)
    agendas_dict = {}
    for tecnico in tecnicos_validos:
        if tecnico in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=tecnico)
            if 'data' in df.columns:
                df['data'] = pd.to_datetime(df['data'], errors='coerce')
            agendas_dict[tecnico] = df
        else:
            print(f"Atenção: técnico {tecnico} não encontrado na planilha agenda.xlsx")
    return agendas_dict


def filtrar_chamados_agendados_futuros(chamados_com_tecnico, agendas_dict):
    hoje = pd.Timestamp.today().normalize()
    ids_agendados = set()

    for tecnico, agenda_df in agendas_dict.items():
        if all(col in agenda_df.columns for col in ['agenda_efetivada', 'id', 'data']):
            # ✅ Conversão explícita da coluna 'data'
            agenda_df['data'] = pd.to_datetime(agenda_df['data'], errors='coerce')

            filtro = (
                agenda_df['agenda_efetivada'].astype(str).str.strip().str.lower().isin(['sim', 'sugerida'])
            ) & (
                agenda_df['data'] > hoje
            )
            ids = agenda_df.loc[filtro, 'id'].dropna().astype(int).tolist()
            ids_agendados.update(ids)

    print(f"IDs agendados e que devem ser removidos dos chamados: {ids_agendados}")

    chamados_filtrados = chamados_com_tecnico[~chamados_com_tecnico['id'].isin(ids_agendados)].copy()
    print(f"Quantidade de chamados após filtro de agendados futuros: {len(chamados_filtrados)}")

    chamados_filtrados.reset_index(drop=True, inplace=True)
    chamados_filtrados.to_excel('chamados.xlsx', index=False)

    return chamados_filtrados

def filtrar_agendas_disponiveis(agendas_dict):
    hoje = pd.to_datetime(datetime.today().date())
    agendas_filtradas = {}

    for tecnico, df in agendas_dict.items():
        if not {'data', 'agenda_efetivada', 'status_agenda'}.issubset(df.columns):
            print(f"Planilha de {tecnico} está faltando colunas necessárias.")
            continue

        df['data'] = pd.to_datetime(df['data'], errors='coerce')
        df['agenda_efetivada'] = df['agenda_efetivada'].astype(str).str.strip().str.lower()
        df['status_agenda'] = df['status_agenda'].astype(str).str.strip().str.lower()

        filtro = (
            (df['data'] > hoje) &
            (df['status_agenda'] == 'liberada') &
            (~df['agenda_efetivada'].isin(['sim', 'sugerida']))
        )

        df_filtrado = df[filtro].copy()

        if df_filtrado.empty:
            print(f"Técnico {tecnico} não possui slots disponíveis.")
        agendas_filtradas[tecnico] = df_filtrado

    return agendas_filtradas

def tecnico_liberado_para_cliente(tecnico, cliente, lib_df):
    try:
        status = lib_df.loc[lib_df['cliente'] == cliente, tecnico].values
        if len(status) == 0:
            return False
        return str(status[0]).strip().lower() == 'liberado'
    except KeyError:
        return False  # técnico não encontrado nas colunas
    

def alocar_chamado(chamado, agendas_disponiveis, tecnicos_liberados, lib_df, alocacoes_cliente_dia):

    model = cp_model.CpModel()
    tempo_necessario = int(chamado['tempo_necessario'])

    tecnicos_validos = [
        t for t in tecnicos_liberados
        if tecnico_liberado_para_cliente(t, chamado['cliente'], lib_df)
        and t in agendas_disponiveis
    ]

    possibilidades = []
    data_aloc = {}
    slots_por_opcao = {}

    for t in tecnicos_validos:
        agenda = agendas_disponiveis[t].copy()
        agenda['data'] = pd.to_datetime(agenda['data'], errors='coerce')

        agenda['agenda_efetivada'] = (
            agenda['agenda_efetivada']
            .fillna('nao')
            .astype(str)
            .str.strip()
            .str.lower()
            .replace({'nan': 'nao', '': 'nao'})
        )

        agenda['id'] = (
            agenda['id']
            .fillna('')
            .astype(str)
            .str.strip()
            .str.lower()
        )

        datas_disponiveis = sorted(agenda['data'].dt.normalize().dropna().unique())

        for data in datas_disponiveis:
            data_normalizada = pd.to_datetime(data).normalize()
            # Verifica se já há técnico alocado para o mesmo cliente na mesma data
            if (chamado['cliente'], data_normalizada.date()) in alocacoes_cliente_dia:
                continue  # Pula essa data

            slots_livres = agenda[
                (agenda['data'].dt.normalize() == data_normalizada) &
                (agenda['agenda_efetivada'] == 'nao') &
                (agenda['id'].isin(['', 'nan', 'none']))
            ].sort_values(by='horario')

            total_slots = len(slots_livres)

            if total_slots >= tempo_necessario:
                var = model.NewBoolVar(f'aloc_{t}_{data.date()}')
                data_aloc[(t, data)] = var
                timestamp = data.timestamp()
                possibilidades.append((var, timestamp, t, data))
                slots_por_opcao[(t, data)] = total_slots - tempo_necessario  # ← slots restantes após alocação

    if not possibilidades:
        return None, None, None  # nenhum técnico/data disponível

    model.AddExactlyOne(var for var, _, _, _ in possibilidades)

    model.Minimize(
        sum(timestamp * var for var, timestamp, _, _ in possibilidades)
    )

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for (t, data), var in data_aloc.items():
            if solver.BooleanValue(var):
                slots_restantes = slots_por_opcao.get((t, data), 0)
                return t, data, slots_restantes

    return None, None, None


def alocar_chamados_adicionais_mesmo_cliente(
    chamado_principal,
    tecnico_alocado,
    data_alocada,
    slots_restantes,
    chamados_pendentes,
    agendas_dict
):
    cliente = chamado_principal['cliente']
    chamados_sugeridos = []

    # Mantém a ordem original dos chamados pendentes
    chamados_cliente = chamados_pendentes[
        (chamados_pendentes['cliente'] == cliente) &
        (chamados_pendentes['id'] != chamado_principal['id'])
    ]

    for _, chamado in chamados_cliente.iterrows():
        tempo_chamado = int(chamado['tempo_necessario'])

        if tempo_chamado <= slots_restantes:
            chamados_sugeridos.append({
                'chamado': chamado,
                'tempo': tempo_chamado
            })
            slots_restantes -= tempo_chamado
        else:
            print(
                f"Sem slots suficientes para o chamado ID {chamado['id']} "
                f"(necessário: {tempo_chamado}, disponíveis: {slots_restantes})."
            )

        if slots_restantes <= 0:
            break

    return chamados_sugeridos

def salvar_agendas_no_excel(agendas_dict, arquivo='agenda.xlsx'):
    with pd.ExcelWriter(arquivo, engine='openpyxl', mode='w') as writer:
        for tecnico, df in agendas_dict.items():
            df.to_excel(writer, sheet_name=tecnico, index=False)
    print(f"Agendas atualizadas foram salvas no arquivo '{arquivo}'.")

def montar_df_alocacoes(tecnico, data, chamados, adicionar_deslocamento=False, slots_totais=None):
    """
    Cria um DataFrame com as linhas que deverão ser aplicadas na agenda
    baseado nos chamados e slots necessários.

    Se adicionar_deslocamento for True e slots_totais for fornecido,
    adiciona uma linha extra para deslocamento (tempo_necessario=1),
    desde que haja pelo menos 1 slot disponível.
    """
    linhas = []

    total_slots_usados = 0
    for chamado in chamados:
        tempo = int(chamado['tempo_necessario'])
        total_slots_usados += tempo

        for _ in range(tempo):
            linhas.append({
                'tecnico': tecnico,
                'data': data,
                'id': chamado['id'],
                'cliente': chamado['cliente'],
                'maquina': chamado.get('maquina', ''),
                'tipo_manutencao': chamado.get('tipo_manutencao', ''),
                'tempo_necessario': tempo,
                'prioridade': chamado.get('prioridade', ''),
                'data_abertura': chamado.get('data_abertura', ''),
                'agenda_efetivada': 'sugerida',
                'status_agenda': 'liberada'
            })

    # Adiciona deslocamento se solicitado e houver slots restantes
    if adicionar_deslocamento and slots_totais is not None:
        slots_restantes = slots_totais - total_slots_usados
        if slots_restantes >= 1:
            linhas.append({
                'tecnico': tecnico,
                'data': data,
                'id': 0,
                'cliente': 'deslocamento',
                'maquina': '',
                'tipo_manutencao': '',
                'tempo_necessario': 1,
                'prioridade': '',
                'data_abertura': '',
                'agenda_efetivada': 'sugerida',
                'status_agenda': 'liberada'
            })

    return pd.DataFrame(linhas)


def atualizar_agenda_com_df(tecnico, data, df_alocacoes, agendas_dict):
    """
    Atualiza a agenda do técnico na data especificada, preenchendo os slots livres
    com os dados do df_alocacoes, na ordem que vierem.
    
    Parâmetros:
    - tecnico: str, nome do técnico
    - data: datetime.date ou datetime.datetime
    - df_alocacoes: pd.DataFrame contendo as linhas a serem alocadas (já com os dados do chamado)
    - agendas_dict: dict com DataFrames das agendas por técnico (modificado in-place)
    """

    agenda_df = agendas_dict[tecnico]

    # Garantir datetime na coluna 'data' e horário em time
    agenda_df['data'] = pd.to_datetime(agenda_df['data'], errors='coerce').dt.date
    if isinstance(data, pd.Timestamp):
        data = data.date()
    elif hasattr(data, 'date') and callable(data.date):
        data = data.date()
    
    # Filtra os slots livres para essa data (status = liberada, agenda_efetivada != 'sim' nem 'sugerida', id vazio)
    mask_livres = (
        (agenda_df['data'] == data) &
        (agenda_df['status_agenda'].str.strip().str.lower() == 'liberada') &
        (~agenda_df['agenda_efetivada'].str.strip().str.lower().isin(['sim', 'sugerida'])) &
        (agenda_df['id'].astype(str).str.strip().isin(['', 'nan', 'none']))
    )

    slots_livres = agenda_df.loc[mask_livres].sort_values(by='horario')

    # Verifica se tem slots livres suficientes
    if len(slots_livres) < len(df_alocacoes):
        print(f"Atenção: número de slots livres ({len(slots_livres)}) insuficiente para alocar {len(df_alocacoes)} chamadas.")
        return False

    # Atualiza a agenda preenchendo os slots livres na ordem com as linhas de df_alocacoes
    indices_slots = slots_livres.index.tolist()

    for i, idx_slot in enumerate(indices_slots[:len(df_alocacoes)]):
        linha = df_alocacoes.iloc[i]

        agenda_df.at[idx_slot, 'id'] = linha['id']
        agenda_df.at[idx_slot, 'cliente'] = linha['cliente']
        agenda_df.at[idx_slot, 'maquina'] = linha['maquina']
        agenda_df.at[idx_slot, 'tipo_manutencao'] = linha['tipo_manutencao']
        agenda_df.at[idx_slot, 'tempo_necessario'] = linha['tempo_necessario']
        agenda_df.at[idx_slot, 'prioridade'] = linha['prioridade']
        agenda_df.at[idx_slot, 'data_abertura'] = linha['data_abertura']
        agenda_df.at[idx_slot, 'agenda_efetivada'] = 'sugerida'
        # status_agenda continua 'liberada'

    # Atualiza o dict com a agenda modificada
    agendas_dict[tecnico] = agenda_df

    return True

def atualizar_chamados_verificacao(arquivo_chamados='chamados_verificacao.xlsx', arquivo_agenda='agenda.xlsx'):
    # Carrega chamados para atualizar
    chamados_verif_df = pd.read_excel(arquivo_chamados)

    # Abre agenda.xlsx com openpyxl via pandas ExcelFile
    xls = pd.ExcelFile(arquivo_agenda)

    alocacoes = []

    for tecnico in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=tecnico)

        # Filtra linhas alocadas (agenda_efetivada == 'sugerida')
        if 'agenda_efetivada' in df.columns:
            alocadas = df[df['agenda_efetivada'].astype(str).str.strip().str.lower() == 'sugerida']
            for _, row in alocadas.iterrows():
                id_chamado = row.get('id')
                data = row.get('data')
                horario = row.get('horario') if 'horario' in row else None

                if pd.isna(id_chamado) or id_chamado == 0 or str(id_chamado).strip() == '':
                    continue  # pula deslocamento ou entradas inválidas
                
                alocacoes.append({
                    'id': int(id_chamado),
                    'tecnico': tecnico,
                    'data': pd.to_datetime(data).date() if not pd.isna(data) else None,
                    'horario': horario
                })

    # Cria dataframe das alocações
    df_alocacoes = pd.DataFrame(alocacoes)

    # Remove duplicados mantendo o primeiro registro (caso algum chamado esteja repetido)
    df_alocacoes = df_alocacoes.drop_duplicates(subset=['id'])

    # Junta as informações no DataFrame de chamados_verif_df pelo 'id'
    chamados_verif_df = chamados_verif_df.merge(df_alocacoes, on='id', how='left')

    # === NOVO BLOCO PARA IDENTIFICAR DUPLICADOS ===
    if 'cliente' in chamados_verif_df.columns and 'data' in chamados_verif_df.columns:
        # Conta quantas vezes cada cliente aparece na mesma data com técnicos diferentes
        contagem = (
            chamados_verif_df
            .dropna(subset=['cliente', 'data', 'tecnico'])  # Remove nulos para a verificação
            .groupby(['cliente', 'data'])['tecnico']
            .nunique()
            .reset_index(name='qtd_tecnicos')
        )

        # Marca cliente/data com mais de 1 técnico como duplicado
        contagem['duplicado'] = contagem['qtd_tecnicos'].apply(lambda x: 'sim' if x > 1 else 'não')

        # Junta de volta com o DataFrame principal
        chamados_verif_df = chamados_verif_df.merge(contagem[['cliente', 'data', 'duplicado']], on=['cliente', 'data'], how='left')
    else:
        chamados_verif_df['duplicado'] = 'não'  # fallback caso faltem colunas

    # Salva atualizado
    chamados_verif_df.to_excel(arquivo_chamados, index=False)
    print(f"Arquivo '{arquivo_chamados}' atualizado com alocações de técnico, data, horário e verificação de duplicidade.")

    


chamados_df, lib_df = carregar_dados('chamados.xlsx', 'liberacao_tecnicos_clientes.xlsx')

# Criar cópia do chamados.xlsx para chamados_verificacao.xlsx
shutil.copyfile('chamados.xlsx', 'chamados_verificacao.xlsx')

todos_os_tecnicos = obter_todos_os_tecnicos_possiveis(lib_df)  # você pode gerar isso a partir do lib_df
agendas_dict = carregar_agendas(todos_os_tecnicos)

alocacoes_cliente_dia = {}

while not chamados_df.empty:
    print(f"\n🔁 Iniciando iteração. Chamados restantes: {len(chamados_df)}")

    # Passo 1: Carregar dados
    chamados_df, lib_df = carregar_dados('chamados.xlsx', 'liberacao_tecnicos_clientes.xlsx')

    # Passo 2: Identificar clientes sem técnico liberado
    clientes = chamados_df['cliente'].unique()
    clientes_sem_tecnico = obter_clientes_sem_tecnico(lib_df, clientes)

    # Passo 3: Separar chamados
    chamados_com_tecnico, chamados_sem_tecnico = separar_chamados_por_tecnico(chamados_df, clientes_sem_tecnico)

    # Passo 4: Atualizar arquivos Excel
    atualizar_chamados_excel(chamados_com_tecnico)
    atualizar_chamados_nao_agendados_excel(chamados_sem_tecnico)

    # Passo 5: Atualizar clientes válidos e técnicos válidos
    clientes_validos = [c for c in clientes if c not in clientes_sem_tecnico]
    clientes_validos = sorted(clientes_validos, key=extrair_numero)
    tecnicos_validos = obter_tecnicos_validos(lib_df, clientes_validos)

    print("Clientes válidos ordenados:", clientes_validos)
    print("Técnicos válidos ordenados:", tecnicos_validos)

    # Passo 6: Carregar agendas dos técnicos válidos
    agendas_iteracao = carregar_agendas(tecnicos_validos)

    # Passo 7: Filtrar chamados já agendados para datas futuras
    chamados_filtrados = filtrar_chamados_agendados_futuros(chamados_com_tecnico, agendas_dict)

    # Passo 8: Verifica se há chamados a serem alocados
    if chamados_filtrados.empty:
        print("✅ Todos os chamados foram alocados ou não há chamados válidos restantes.")
        break

    # Continua com o primeiro chamado
    primeiro_chamado = chamados_filtrados.iloc[0]

    # Filtra agendas disponíveis conforme regras
    agendas_disponiveis = filtrar_agendas_disponiveis(agendas_dict)

    # Técnicos liberados para o cliente do chamado
    tecnicos_liberados = tecnicos_validos  # ou use outra lista se quiser filtrar mais

    tecnico_alocado, data_alocada, slots_livres_restantes = alocar_chamado(
        primeiro_chamado,
        agendas_disponiveis,
        tecnicos_liberados,
        lib_df,
        alocacoes_cliente_dia
    )

    print(f"Técnico: {tecnico_alocado}, Data: {data_alocada}, Slots livres naquele dia após alocação: {slots_livres_restantes}")

    if tecnico_alocado and data_alocada:
        print(f"Chamado ID {primeiro_chamado['id']} (cliente: {primeiro_chamado['cliente']}) alocado para técnico {tecnico_alocado} na data {data_alocada}")

        # Busca e aloca outros chamados do mesmo cliente
        chamados_adicionais = alocar_chamados_adicionais_mesmo_cliente(
            primeiro_chamado,
            tecnico_alocado,
            data_alocada,
            slots_livres_restantes,
            chamados_filtrados,
            agendas_dict
        )

        if chamados_adicionais:
            print(f"{len(chamados_adicionais)} chamado(s) adicional(is) do cliente {primeiro_chamado['cliente']} também alocado(s):")
            for c in chamados_adicionais:
                chamado = c['chamado']
                print(f"  - ID {chamado['id']} (tempo: {chamado['tempo_necessario']} slots)")
        else:
            print("Nenhum outro chamado adicional pôde ser alocado.")

        # Controlar limite de slots para não extrapolar
        total_slots = int(primeiro_chamado['tempo_necessario'])
        chamados_selecionados = []

        for c in chamados_adicionais:
            tempo = int(c['chamado']['tempo_necessario'])
            if total_slots + tempo <= total_slots + slots_livres_restantes:
                chamados_selecionados.append(c['chamado'])
                total_slots += tempo
            else:
                print(f"Chamado ID {c['chamado']['id']} não cabe nos slots restantes ({slots_livres_restantes}) e foi descartado.")

        alocacoes_cliente_dia[(primeiro_chamado['cliente'], data_alocada.date())] = tecnico_alocado

        # Calcula o total de slots usados pelo primeiro chamado e pelos adicionais selecionados
        total_slots_usados = int(primeiro_chamado['tempo_necessario']) + sum(int(c['tempo_necessario']) for c in chamados_selecionados)

        # Verifica se há espaço para o deslocamento (1 slot)
        adicionar_deslocamento = (slots_livres_restantes - (total_slots_usados - int(primeiro_chamado['tempo_necessario']))) >= 1

        # Monta lista final para alocar (primeiro chamado + adicionais)
        chamados_para_alocar = [primeiro_chamado] + chamados_selecionados

        # Monta dataframe com as linhas para a agenda, incluindo deslocamento se couber
        df_alocacoes = montar_df_alocacoes(
            tecnico_alocado,
            data_alocada,
            chamados_para_alocar,
            adicionar_deslocamento=adicionar_deslocamento,
            slots_totais=slots_livres_restantes + total_slots_usados
        )

        # Atualiza agenda do técnico e data na memória
        atualizar_agenda_com_df(tecnico_alocado, data_alocada, df_alocacoes, agendas_dict)

        # Salva agenda atualizada no Excel
        salvar_agendas_no_excel(agendas_dict, arquivo='agenda.xlsx')

    else:
        print("❌ Nenhuma alocação possível para o chamado.")

else:
    print("🏁 Todos os chamados foram processados ou não há dados no DataFrame.")



# Chama a função no final do script:
atualizar_chamados_verificacao()






🔁 Iniciando iteração. Chamados restantes: 200
Clientes válidos ordenados: ['cliente_1', 'cliente_2', 'cliente_3', 'cliente_4', 'cliente_5', 'cliente_6', 'cliente_7', 'cliente_8', 'cliente_10', 'cliente_11', 'cliente_12', 'cliente_13', 'cliente_14', 'cliente_15', 'cliente_16', 'cliente_17', 'cliente_18', 'cliente_19', 'cliente_20', 'cliente_21', 'cliente_22', 'cliente_23', 'cliente_24', 'cliente_25', 'cliente_26', 'cliente_27', 'cliente_28', 'cliente_29', 'cliente_30']
Técnicos válidos ordenados: ['tecnico_1', 'tecnico_2', 'tecnico_3', 'tecnico_4', 'tecnico_5']
IDs agendados e que devem ser removidos dos chamados: set()
Quantidade de chamados após filtro de agendados futuros: 193
Técnico: tecnico_2, Data: 2025-09-29 00:00:00, Slots livres naquele dia após alocação: 6
Chamado ID 31 (cliente: cliente_1) alocado para técnico tecnico_2 na data 2025-09-29 00:00:00
Sem slots suficientes para o chamado ID 92 (necessário: 3, disponíveis: 2).
3 chamado(s) adicional(is) do cliente cliente_1 tamb